In [1]:
import pandas as pd
import numpy as np

In [2]:
moex = 'https://www.moex.com/'
init = 'a20'  # 4 квартал 2011 г.
urls = (('a685', 'a1127', 'a1244', 'a1465'), ('a1688', 'a1823', 'a2045', 'a2261'),
        ('a2474', 'a2702', 'a2813', 'a2987'), ('a3048', 'a3177', 'a3369', 'a3503'),
        ('a3601', 'a3691', 'a3794', 'a3882'), ('a4027', 'a4094', 'a4184', 'a4258'),
        ('a4318', 'a4377', 'a4420', 'a6401'), ('a6861', 'a6997', 'a7080', 'a7150'),
        ('a7246', 'a7305', 'a7396', 'a7605'))  # квартальные за 2012-2020 гг.

In [3]:
years = range(2012, 2021)
quarters = range(1, 5)

In [4]:
def parser(url):
    df = pd.read_html(url, index_col=0, thousands=' ', decimal=',')[0]
    df.columns = ['name', 'category', 'id', 'volume', 'price', 'cap']
    df.index.name = 'code'
    return df

In [5]:
def string_handler(string):
    if type(string) == str:
        x = bytes(string, 'latin1')
        x = x.replace(b'\xa0', b'')
        x = x.replace(b',', b'.')
        x = x.decode('latin1')
        x = x.replace(' ', '', 10)
        return x
    return string

In [6]:
df = parser(moex + init)

for i, year in enumerate(years):
    for j, quarter in enumerate(quarters):
        df = df.merge(parser(moex + urls[i][j]), how='outer',
                      left_index=True, right_index=True,
                      suffixes=(None, '_' + str(year) + '_' + str(quarter)))

In [7]:
for col in df.columns:
    if 'cap' in col:
        df[col] = df[col].apply(string_handler).astype(float)
    elif 'volume' in col:
        df[col] = pd.to_numeric(df[col].apply(string_handler), errors='coerce')

In [13]:
names = dict.fromkeys(df.index.values)
for col in df.columns:
    if 'name' in col:
        for idx in names:
            if df[col].loc[idx]:
                names[idx] = df[col].loc[idx]
names

{'ABBN': nan,
 'ABRD': 'ПАО "Абрау – Дюрсо"',
 'ACKO': 'ПАО "АСКО-СТРАХОВАНИЕ"',
 'AESL': nan,
 'AFKS': 'ПАО АФК "Система"',
 'AFLT': 'ПАО "Аэрофлот-российские авиалинии"',
 'AGRE': nan,
 'AKHA': nan,
 'AKRN': 'ПАО "Акрон"',
 'ALBK': 'ПАО "Бест Эффортс Банк"',
 'ALNU': 'ПАО "АЛРОСА-Нюрба"',
 'ALRS': 'АК "АЛРОСА" (ПАО)',
 'AMEZ': 'ПАО "Ашинский метзавод"',
 'ANGA': nan,
 'APTK': 'ПАО "Аптечная сеть 36,6"',
 'AQUA': 'ПАО "Русская Аквакультура"',
 'ARMD': nan,
 'ARSA': 'ПАО "УК "Арсагера"',
 'ARSB': nan,
 'ARSBP': nan,
 'ASSB': 'ПАО "Астраханская энергосбытовая компания"',
 'AVAN': 'ПАО АКБ "АВАНГАРД"',
 'AVAZ': nan,
 'AVAZP': nan,
 'AZKM': nan,
 'BACT': nan,
 'BANE': 'ПАО АНК "Башнефть"',
 'BANE-001D': nan,
 'BANE-002D': nan,
 'BANE-004D': nan,
 'BANE-005D': nan,
 'BANEP': 'ПАО АНК "Башнефть"',
 'BANEP-001D': nan,
 'BANEP-002D': nan,
 'BANEP-003D': nan,
 'BANEP-004D': nan,
 'BANEP-005D': nan,
 'BEGY': nan,
 'BEGYP': nan,
 'BELU': 'ПАО "Белуга Групп"',
 'BGDE': nan,
 'BISV': 'ПАО "Башинфо

In [ ]:
#df.to_csv('market_cap.csv', sep=';', decimal=',')